## Librerias

In [1]:
import os
import sys
import configparser
import logging
import pandas as pd
import geopandas as gpd
import numpy as np
import glob
import datetime as dt
import pytz
import datetime
import math
import time
from sklearn.neighbors import BallTree
from math import sin, cos, sqrt, atan2, radians
import psycopg2

import multiprocessing as multi_p 
from multiprocessing import Pool
from operator import itemgetter

global DISTANCE
global INTERVAL
global M
global T_START
global T_END
global T_START_INF  
global T_END_INF
global OUTPUTPATH
global GAMMA

## Funciones de apoyo

#### Consultas a la Base de de datos

In [2]:
def connectionDB():
    """ Get connection to the database"""
    conn_string = "dbname='' port='' user='' password='' host=''";
    #print ("Connecting to database ->{}".format(conn_string))
    conn = psycopg2.connect(conn_string);
    #print("Conexion buena")
    return conn

def getInfected(t_start,t_end):
    """ This method gets the infected list from the database"""
    conn=connectionDB()
    cursor = conn.cursor();
    query1='''
          SELECT device_id FROM register INNER JOIN infected
          ON register.user_id=infected.user_id
          WHERE register.datetime>=
    '''
    query2 = '''
          AND register.datetime<=
    '''
    query=query1+" "+"'"+t_start+"'"+" "+query2+" "+"'"+t_end+"'"
    cursor.execute(query);
    rows = cursor.fetchall()
    df = pd.read_sql(query, conn) # storing the query in a pandas dataframe
    infected=list(set(df['device_id'].to_list()))
    return infected

def getUserDeviceID():
    conn = connectionDB()
    cursor = conn.cursor()
    query = """
    SELECT distinct device_id,user_id
    FROM register;
    """
    cursor.execute(query);
    rows = cursor.fetchall()
    df = pd.read_sql(query, conn) # storing the query in a pandas dataframe
    conn.close()
    return df

#### Consulta de registro GPS

In [3]:
def getRecords(path):
    records = pd.read_csv(path)
    del records['user_id']
    del records['date']
    records = records[records['type'] == 'LOCATION']
    del records['type']
    records = records.drop(records[records['x'] == records['y']].index)
    records = records.dropna()
    records = records.sort_values(by=['datetime']).reset_index(drop=True)
    #records['intervalo'] = records['datetime'].apply(lambda x: label_interval(x,lista_intervalos))
    return records

#### Construccion de Funciones para manejo de intervalos

In [4]:
def getIntervals(t_start,t_end,interval):
    """ This method gets the list of dates between t_start and t_end with an interval in minutes
        Input:
        ------
           @t_start : string
              start time
           @t_end : string
              end time
           @interval : int
              interval between dates
        Output:
        -------
           @list_dates : list
              list of intervals dates between t_start and t_end with an interval
    """
    timestamp_start = pd.Timestamp(t_start)
    timestamp_end = pd.Timestamp(t_end)
    nro_elements=int(int((timestamp_end-timestamp_start).total_seconds())/(60*interval))
    list_dates = []
    list_dates.append(t_start)
    for i in range(nro_elements):
        date_time_str = list_dates[i]
        Current_Date = dt.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
        Next_Date = Current_Date + datetime.timedelta(minutes=interval)
        list_dates.append(str(Next_Date))
    intervalos = []
    for j in range(len(list_dates)-1):
        intervalos.append([j,list_dates[j],list_dates[j+1]])
    return intervalos

In [5]:
def label_interval(x,lista_intervalos):
    for i in lista_intervalos:
        if x >= i[1] and x <= i[2]:
            return i[0]
    return -1

#### Funcion de procesamiento de intervalos (corren en paralelo)

In [6]:
def getTrazaTimestamp(array, send_end):
    
    def distanceL(punto1, punto2):
        lat1, lon1 = punto1
        lat2, lon2 = punto2
        dlon = abs(lon2 - lon1) * 110139
        dlat = abs(lat2 - lat1) * 110139
        return dlon + dlat

    def get_nearest(infected_coordinates, uninfected_coordinates, d):
        # Create tree from the GPS coordinates of uninfected users
        tree = BallTree(uninfected_coordinates, leaf_size=15, metric='pyfunc', func=distanceL)
        indices,distances=tree.query_radius(infected_coordinates, r=d,return_distance=True)
        indices=indices.transpose()
        distances=distances.transpose()
        return indices,distances

    def nearest_neighbor(left_gdf, right_gdf, distance):
        right_gdf = right_gdf.copy().reset_index(drop=True)
        #left_gdf = left_gdf.copy().reset_index(drop=True)
        
        left_points = np.array(list(map(lambda x,y: [y,x], left_gdf['x'].to_list(),left_gdf['y'].to_list())))
        right_points = np.array(list(map(lambda x,y: [y,x], right_gdf['x'].to_list(),right_gdf['y'].to_list())))
        
        indices,meters = get_nearest(infected_coordinates=left_points, uninfected_coordinates=right_points , d=distance)
        infected=[];uninfected=[]
        for i in range(len(left_gdf)):
            list_indices=[];list_distances=[]
            list_indices = [i] * len(indices[i])
            points_infected = left_gdf.loc[list_indices]
            list_distances=(meters[i]).tolist()
            points_infected = points_infected.reset_index(drop=True)
            infected.append(points_infected.to_numpy())
            points_uninfected = right_gdf.loc[indices[i]]
            points_uninfected = points_uninfected.reset_index(drop=True)
            points_uninfected['distance']=list_distances
            uninfected.append(points_uninfected.to_numpy())
        array_infected = np.concatenate(infected,axis=0)
        array_uninfected = np.concatenate(uninfected,axis=0)
        
        events_infected = pd.DataFrame(array_infected,columns=["time_i","device_i","lon_i","lat_i"])
        #events_infected = events_infected.drop('geometry', axis=1)
        events_uninfected= pd.DataFrame(array_uninfected,columns=["time_j","device_j","lon_j","lat_j","distance"])
        #events_uninfected = events_uninfected.drop('geometry', axis=1)
        #events_uninfected = events_uninfected.rename(columns={'geometry': 'closest_stop_geom'})
        GPSevents = events_infected.join(events_uninfected)
        return GPSevents

    #################################################################################################################
    #[interval_i,registros_gps,list_infected,DISTANCE]
    
    interval_i = array[0][0]
    min_f = array[0][1]
    max_f = array[0][2]
    dataset = array[1]
    list_infected = array[2]
    d = array[3]
    
    traza_uninfected = dataset[(dataset['datetime']>=min_f) & (dataset['datetime']<max_f) & (~dataset['device_id'].isin(list_infected))].reset_index(drop=True)
    traza_infected = dataset[(dataset['datetime']>=min_f) & (dataset['datetime']<max_f) & (dataset['device_id'].isin(list_infected))].reset_index(drop=True)
    
    del dataset
    del list_infected
    # drop duplicates in terval where (user, lat, lon)
    traza_uninfected.drop_duplicates(subset =['device_id','x','y'], keep = 'first' , inplace = True, ignore_index=True)
    traza_infected.drop_duplicates(subset =['device_id','x','y'], keep = 'first' , inplace = True, ignore_index=True) 

    try:
        events_gps = nearest_neighbor(traza_infected, traza_uninfected, d)
        if(len(events_gps)!=0):
            #list_gpsevents.append(events_gps.reset_index(drop=True))
            events_gps =  events_gps.reset_index(drop=True)
        else:
            events_gps=pd.DataFrame()
            #list_gpsevents.append(events_gps)
    except:
        events_gps=pd.DataFrame()
        #list_gpsevents.append(events_gps)
    
    print("  procesando elemento",interval_i," -- [",len(traza_uninfected),",",len(traza_infected),"] -- ",len(events_gps))
    del traza_infected
    del traza_uninfected
    send_end.send([interval_i,events_gps])

#### Calculo de probabilidades

In [7]:
def probaContagius(lat1,lon1,lat2,lon2,M):
    """
        This method computes the distance lof latitudes and longitudes
        of two GPS coordinates in decimeters to return the contagius probability
        of the co-location represented by (lat1,lon1) and (lat2,lon2).
        Input:
        ------
           @lat1 : float
               Latitude of the first GPS coordinate (infected)
           @lat1 : float
               Longitude of the first GPS coordinate (infected)
           @lat2 : float
               Latitude of the second GPS coordinate
           @lat3 : float
               Longitude of the second GPS coordinate
           @M : numpy matrix
               Matrix containing the contagius probability based on distance
        Output:
        -------
           @proba : float
               Probabilty of being infected.
    """
    #GAMMA = 0.02
    dlon = abs(lon2 - lon1) * 10000
    dlat = abs(lat2 - lat1) * 10000
    distance_Y = int(round(dlon, 0))
    distance_X = int(round(dlat, 0))
    proba = 0
    if ( (distance_X>=0 and distance_X<300) and (distance_Y>=0 and distance_Y<300) ):
        proba = GAMMA * M[distance_X][distance_Y]
    return proba

def componeProbs(p,p_prime):
    """
        This method compose two independent probabilities
        Input:
        ------
           @p : float
               Last computed probability
           @p_prime : float
               New computed probability
        Output:
        -------
           @proba : float
               Probability composition of the new and last probabilities
    """
    return p + p_prime * (1-p)

## Lectura de parametros y ejecucion

In [8]:
#config_file_ = sys.argv[1]
config_file_ = "config.cfg"

if __name__ == "__main__":
    if len(sys.argv)<=1:
        print("ERROR: You need to specify the path of the config file")
    else:
        now = dt.datetime.now()
        
        cfgName = config_file_
        config = configparser.ConfigParser()
        config.read(cfgName)
        logging.info("Reading configuration")
        print (config.sections())
        T_START = (config.get('parameters','t_start')).strip("'")
        T_END = (config.get('parameters','t_end')).strip("'")
        T_START_INF = (config.get('parameters','t_start_infected')).strip("'")
        T_END_INF = (config.get('parameters','t_end_infected')).strip("'")
        DISTANCE = int(config.get('parameters','distance'))
        INTERVAL = int(config.get('parameters','interval'))
        GAMMA  = float(config.get('parameters','gamma'))
        MATRIX = (config.get('path','matrix')).strip("'")
        OUTPUTPATH = (config.get('path','outputFilePath')).strip("'")
        INPUTPATH = (config.get('path','inputFilePath')).strip("'")
        M = np.load(MATRIX)

['path', 'parameters']


In [9]:
([T_START, T_END],[T_START_INF,T_END_INF],
 DISTANCE, INTERVAL, GAMMA,
 [MATRIX], [INPUTPATH,OUTPUTPATH])

(['2020-05-10 00:00:00', '2020-05-26 00:00:00'],
 ['2020-05-10 00:00:00', '2020-05-26 00:00:00'],
 40,
 5,
 0.12,
 ['matrizproba_n_10000000_dist_15_sigma_50_rng_300.npy'],
 ['/home/datos_covid/procesados/', 'events_dist_15_sigma_50_rng_300/'])

In [10]:
# Creacion de ficheros(RESULTADOS)
path_general = os.getcwd()
path = path_general+"/"+OUTPUTPATH
try: os.mkdir(path)
except OSError: print ("Creation of the directory %s failed(maybe it already exists)" % path)
else: print ("Successfully created the directory %s " % path)

Creation of the directory /home/jbarrera/covid19_prototype/modelo_v1/events_dist_15_sigma_50_rng_300/ failed(maybe it already exists)


In [11]:
list_infected = getInfected(T_START_INF,T_END_INF)
len(list_infected)

841

In [12]:
# LISTA DE ARCHIVOS A LEER
files_gps = []
for elem in glob.glob(INPUTPATH+"gps_*.csv"):
    if( elem >= INPUTPATH+"gps_"+T_START[0:4]+T_START[5:7]+T_START[8:10] 
       and  elem < INPUTPATH+"gps_"+T_END[0:4]+T_END[5:7]+T_END[8:10]):
        files_gps.append(elem)
files_gps = np.sort(files_gps)
print("Numero de archivos:",len(files_gps))

Numero de archivos: 84


In [13]:
files_gps

array(['/home/datos_covid/procesados/gps_20200510_lote_00.csv',
       '/home/datos_covid/procesados/gps_20200510_lote_01.csv',
       '/home/datos_covid/procesados/gps_20200510_lote_02.csv',
       '/home/datos_covid/procesados/gps_20200510_lote_03.csv',
       '/home/datos_covid/procesados/gps_20200510_lote_04.csv',
       '/home/datos_covid/procesados/gps_20200510_lote_05.csv',
       '/home/datos_covid/procesados/gps_20200511_lote_00.csv',
       '/home/datos_covid/procesados/gps_20200511_lote_01.csv',
       '/home/datos_covid/procesados/gps_20200511_lote_02.csv',
       '/home/datos_covid/procesados/gps_20200511_lote_03.csv',
       '/home/datos_covid/procesados/gps_20200511_lote_04.csv',
       '/home/datos_covid/procesados/gps_20200511_lote_05.csv',
       '/home/datos_covid/procesados/gps_20200512_lote_00.csv',
       '/home/datos_covid/procesados/gps_20200512_lote_01.csv',
       '/home/datos_covid/procesados/gps_20200512_lote_02.csv',
       '/home/datos_covid/procesados/gps

In [14]:
## CREAMOS LISTA DE INTERVALOS A ANALIZAR [fecha_inicio,fecha_fin>
intervalos=getIntervals(T_START,T_END,INTERVAL)
print("Numero de intervalos:",len(intervalos))

Numero de intervalos: 4608


In [15]:
encounters = []

# leemos cada archivo
df_last_interval = pd.DataFrame()
for file in files_gps:
    registros_gps = getRecords(file)
    print(file)
    
    #lista_intevalos = np.unique(registros_gps['intervalo'])
    min_f =  label_interval(min(registros_gps['datetime']),intervalos)
    max_f =  label_interval(max(registros_gps['datetime']),intervalos)
    lista_intervalos = []
    for e in range(min_f,max_f+1):
        lista_intervalos.append(intervalos[e])
    
    if file != files_gps[-1]:
        if len(df_last_interval) != 0:
            registros_gps = pd.concat([df_last_interval, registros_gps])
            
        df_last_interval = registros_gps[(registros_gps['datetime']>=lista_intervalos[-1][1]) & (registros_gps['datetime']<lista_intervalos[-1][2])].reset_index(drop=True)
        lista_intervalos = lista_intervalos[:-1]
    else:
        lista_intervalos = lista_intervalos
        
    pipe_list = []
    for interval_i  in lista_intervalos:
        recv_end, send_end = multi_p.Pipe(False)
        p = multi_p.Process(target=getTrazaTimestamp,args=([interval_i,registros_gps,list_infected,DISTANCE],send_end))
        p.start()
        pipe_list.append(recv_end)
    parallel_result = [x.recv() for x in pipe_list]  
    p.join()    
    p.terminate()    
    encounters = encounters + parallel_result
    

/home/datos_covid/procesados/gps_20200510_lote_00.csv
  procesando elemento 4  -- [ 46105 , 4 ] --  0
  procesando elemento 11  -- [ 46072 , 10 ] --  8
  procesando elemento 15  -- [ 45634 , 8 ] --  2
  procesando elemento 20  -- [ 45575 , 8 ] --  0
  procesando elemento 24  -- [ 45998 , 16 ] --  10
  procesando elemento 9  -- [ 45544 , 9 ] --  4
  procesando elemento 16  -- [ 45590 , 10 ] --  5
  procesando elemento 18  -- [ 45707 , 10 ] --  3
  procesando elemento 29  -- [ 44668 , 11 ] --  6
  procesando elemento 8  -- [ 45848 , 8 ] --  2
  procesando elemento 22  -- [ 45403 , 8 ] --  4
  procesando elemento 30  -- [ 45145 , 7 ] --  1
  procesando elemento 3  -- [ 45825 , 4 ] --  0
  procesando elemento 17  -- [ 45420 , 10 ] --  1
  procesando elemento 27  -- [ 45279 , 6 ] --  2
  procesando elemento 23  -- [ 45641 , 10 ] --  4
  procesando elemento 1  -- [ 46244 , 6 ] --  0
  procesando elemento 28  -- [ 45319 , 11 ] --  10
  procesando elemento 26  -- [ 45453 , 9 ] --  4
  procesan

  procesando elemento 157  -- [ 31159 , 6 ] --  2
  procesando elemento 165  -- [ 35508 , 10 ] --  6
  procesando elemento 164  -- [ 33570 , 11 ] --  13
  procesando elemento 167  -- [ 36521 , 21 ] --  11
  procesando elemento 171  -- [ 37004 , 16 ] --  8
  procesando elemento 166  -- [ 36609 , 14 ] --  5
  procesando elemento 159  -- [ 32049 , 8 ] --  1
  procesando elemento 175  -- [ 36767 , 15 ] --  9
  procesando elemento 169  -- [ 37145 , 20 ] --  11
  procesando elemento 186  -- [ 38276 , 14 ] --  4
  procesando elemento 173  -- [ 37078 , 13 ] --  2
  procesando elemento 184  -- [ 37872 , 17 ] --  7
  procesando elemento 168  -- [ 37276 , 16 ] --  4
  procesando elemento 172  -- [ 36673 , 15 ] --  7
  procesando elemento 182  -- [ 37967 , 17 ] --  9
  procesando elemento 176  -- [ 36962 , 14 ] --  5
  procesando elemento 181  -- [ 37730 , 13 ] --  5
  procesando elemento 187  -- [ 38409 , 15 ] --  2
  procesando elemento 174  -- [ 36519 , 11 ] --  3
  procesando elemento 178  -- 

  procesando elemento 321  -- [ 41447 , 6 ] --  1
  procesando elemento 289  -- [ 41387 , 5 ] --  7
  procesando elemento 325  -- [ 40825 , 7 ] --  1
  procesando elemento 317  -- [ 41808 , 14 ] --  2
  procesando elemento 295  -- [ 41391 , 9 ] --  3
/home/datos_covid/procesados/gps_20200511_lote_01.csv
  procesando elemento 377  -- [ 20641 , 7 ] --  1
  procesando elemento 372  -- [ 22401 , 7 ] --  4
  procesando elemento 349  -- [ 32622 , 11 ] --  5
  procesando elemento 382  -- [ 19247 , 6 ] --  1
  procesando elemento 364  -- [ 25615 , 6 ] --  6
  procesando elemento 370  -- [ 23228 , 9 ] --  0
  procesando elemento 374  -- [ 21844 , 9 ] --  3
  procesando elemento 342  -- [ 36992 , 10 ] --  7
  procesando elemento 334  -- [ 39703 , 11 ] --  8
  procesando elemento 363  -- [ 26045 , 9 ] --  3
  procesando elemento 355  -- [ 30297 , 6 ] --  2
  procesando elemento 353  -- [ 31458 , 7 ] --  1
  procesando elemento 378  -- [ 20587 , 7 ] --  3
  procesando elemento 381  -- [ 19575 , 9 

  procesando elemento 481  -- [ 41606 , 10 ] --  5
  procesando elemento 470  -- [ 41302 , 8 ] --  2
  procesando elemento 501  -- [ 42903 , 14 ] --  0
  procesando elemento 459  -- [ 39093 , 8 ] --  0
  procesando elemento 485  -- [ 41754 , 5 ] --  2
  procesando elemento 492  -- [ 41727 , 12 ] --  4
  procesando elemento 496  -- [ 41556 , 12 ] --  2
  procesando elemento 479  -- [ 41022 , 7 ] --  2
  procesando elemento 461  -- [ 38965 , 7 ] --  0
  procesando elemento 465  -- [ 40093 , 9 ] --  6
  procesando elemento 472  -- [ 41316 , 5 ] --  0
  procesando elemento 498  -- [ 42791 , 14 ] --  2
  procesando elemento 475  -- [ 40980 , 9 ] --  1
  procesando elemento 483  -- [ 41094 , 7 ] --  3
  procesando elemento 497  -- [ 42004 , 11 ] --  2
  procesando elemento 491  -- [ 42128 , 11 ] --  6
  procesando elemento 500  -- [ 43636 , 10 ] --  1
  procesando elemento 493  -- [ 41754 , 13 ] --  4
  procesando elemento 499  -- [ 42684 , 9 ] --  2
  procesando elemento 486  -- [ 41846 , 7

  procesando elemento 645  -- [ 29741 , 11 ] --  1
  procesando elemento 664  -- [ 21474 , 8 ] --  0
  procesando elemento 627  -- [ 39019 , 11 ] --  1
  procesando elemento 617  -- [ 43310 , 16 ] --  6
  procesando elemento 635  -- [ 34928 , 7 ] --  1
  procesando elemento 619  -- [ 42094 , 17 ] --  2
  procesando elemento 622  -- [ 41792 , 14 ] --  1
  procesando elemento 655  -- [ 24719 , 5 ] --  0
  procesando elemento 624  -- [ 40620 , 13 ] --  5
  procesando elemento 632  -- [ 37415 , 13 ] --  1
  procesando elemento 621  -- [ 42223 , 13 ] --  2
  procesando elemento 647  -- [ 28509 , 9 ] --  0
  procesando elemento 641  -- [ 32037 , 7 ] --  0
  procesando elemento 613  -- [ 43714 , 17 ] --  6
  procesando elemento 628  -- [ 38922 , 12 ] --  3
  procesando elemento 652  -- [ 25994 , 7 ] --  0
  procesando elemento 643  -- [ 30649 , 8 ] --  0
  procesando elemento 615  -- [ 43223 , 16 ] --  2
  procesando elemento 618  -- [ 42568 , 17 ] --  5
  procesando elemento 639  -- [ 33232 

...

  procesando elemento 3897  -- [ 29201 , 27 ] --  0
  procesando elemento 3895  -- [ 28224 , 46 ] --  1
  procesando elemento 3918  -- [ 34424 , 28 ] --  4
  procesando elemento 3899  -- [ 29868 , 28 ] --  0
  procesando elemento 3919  -- [ 34482 , 31 ] --  4
  procesando elemento 3913  -- [ 33830 , 28 ] --  2
  procesando elemento 3901  -- [ 30711 , 27 ] --  3
  procesando elemento 3920  -- [ 34619 , 28 ] --  4
  procesando elemento 3911  -- [ 33476 , 26 ] --  1
  procesando elemento 3902  -- [ 30375 , 26 ] --  1
  procesando elemento 3905  -- [ 31620 , 24 ] --  2
  procesando elemento 3903  -- [ 31091 , 55 ] --  16
  procesando elemento 3914  -- [ 34115 , 27 ] --  4
  procesando elemento 3915  -- [ 33991 , 31 ] --  1
  procesando elemento 3916  -- [ 34519 , 52 ] --  5
  procesando elemento 3917  -- [ 34081 , 31 ] --  1
  procesando elemento 3907  -- [ 32827 , 24 ] --  3
  procesando elemento 3909  -- [ 32505 , 26 ] --  2
  procesando elemento 3910  -- [ 33110 , 66 ] --  9
  procesand

  procesando elemento 4058  -- [ 38422 , 63 ] --  11
  procesando elemento 4077  -- [ 35189 , 42 ] --  5
  procesando elemento 4076  -- [ 35097 , 32 ] --  6
  procesando elemento 4050  -- [ 38670 , 34 ] --  2
  procesando elemento 4063  -- [ 37488 , 35 ] --  7
  procesando elemento 4053  -- [ 38076 , 35 ] --  4
  procesando elemento 4078  -- [ 34962 , 33 ] --  8
  procesando elemento 4075  -- [ 35318 , 37 ] --  8
  procesando elemento 4081  -- [ 33896 , 31 ] --  6
  procesando elemento 4074  -- [ 36365 , 56 ] --  12
  procesando elemento 4085  -- [ 32472 , 26 ] --  5
  procesando elemento 4083  -- [ 32941 , 32 ] --  7
  procesando elemento 4059  -- [ 37812 , 33 ] --  9
  procesando elemento 4051  -- [ 38180 , 36 ] --  5
  procesando elemento 4055  -- [ 37736 , 39 ] --  9
  procesando elemento 4045  -- [ 39111 , 41 ] --  5
  procesando elemento 4048  -- [ 38294 , 36 ] --  4
  procesando elemento 4033  -- [ 39238 , 38 ] --  7
  procesando elemento 4037  -- [ 38438 , 39 ] --  5
  procesan

  procesando elemento 4209  -- [ 30654 , 26 ] --  4
  procesando elemento 4208  -- [ 30473 , 25 ] --  2
  procesando elemento 4215  -- [ 32455 , 26 ] --  4
  procesando elemento 4212  -- [ 31881 , 27 ] --  3
  procesando elemento 4206  -- [ 30914 , 50 ] --  36
  procesando elemento 4214  -- [ 33017 , 31 ] --  7
  procesando elemento 4211  -- [ 31073 , 27 ] --  7
  procesando elemento 4216  -- [ 32203 , 35 ] --  7
  procesando elemento 4219  -- [ 33242 , 30 ] --  6
  procesando elemento 4226  -- [ 34031 , 36 ] --  2
  procesando elemento 4213  -- [ 32026 , 30 ] --  7
  procesando elemento 4224  -- [ 33536 , 29 ] --  9
  procesando elemento 4223  -- [ 33502 , 28 ] --  2
  procesando elemento 4220  -- [ 33605 , 28 ] --  9
  procesando elemento 4225  -- [ 33837 , 27 ] --  5
  procesando elemento 4217  -- [ 32509 , 21 ] --  2
  procesando elemento 4227  -- [ 33932 , 31 ] --  6
  procesando elemento 4222  -- [ 33260 , 32 ] --  5
  procesando elemento 4221  -- [ 32863 , 24 ] --  1
  procesand

  procesando elemento 4344  -- [ 38710 , 36 ] --  1
  procesando elemento 4367  -- [ 34850 , 29 ] --  13
  procesando elemento 4373  -- [ 33040 , 31 ] --  1
  procesando elemento 4370  -- [ 33637 , 36 ] --  6
  procesando elemento 4362  -- [ 36431 , 36 ] --  7
  procesando elemento 4363  -- [ 36030 , 33 ] --  3
  procesando elemento 4360  -- [ 36767 , 33 ] --  4
  procesando elemento 4368  -- [ 34933 , 35 ] --  9
  procesando elemento 4364  -- [ 35917 , 35 ] --  4
  procesando elemento 4365  -- [ 36029 , 35 ] --  11
  procesando elemento 4371  -- [ 33667 , 30 ] --  7
  procesando elemento 4356  -- [ 37588 , 29 ] --  4
  procesando elemento 4354  -- [ 37656 , 31 ] --  0
  procesando elemento 4359  -- [ 37007 , 35 ] --  2
  procesando elemento 4369  -- [ 34240 , 45 ] --  9
/home/datos_covid/procesados/gps_20200525_lote_01.csv
  procesando elemento 4392  -- [ 24834 , 22 ] --  6
  procesando elemento 4380  -- [ 9883 , 9 ] --  0
  procesando elemento 4396  -- [ 23196 , 23 ] --  6
  procesan

  procesando elemento 4515  -- [ 34896 , 33 ] --  4
  procesando elemento 4523  -- [ 35196 , 36 ] --  7
  procesando elemento 4525  -- [ 35229 , 32 ] --  5
  procesando elemento 4509  -- [ 34538 , 35 ] --  6
  procesando elemento 4522  -- [ 34984 , 30 ] --  4
  procesando elemento 4526  -- [ 35165 , 35 ] --  3
  procesando elemento 4530  -- [ 34698 , 30 ] --  3
  procesando elemento 4531  -- [ 34975 , 35 ] --  3
  procesando elemento 4535  -- [ 35279 , 32 ] --  9
  procesando elemento 4528  -- [ 34924 , 34 ] --  3
  procesando elemento 4524  -- [ 35222 , 40 ] --  7
  procesando elemento 4529  -- [ 35305 , 34 ] --  6
  procesando elemento 4521  -- [ 35394 , 33 ] --  4
  procesando elemento 4537  -- [ 35514 , 33 ] --  6
  procesando elemento 4534  -- [ 35368 , 31 ] --  4
  procesando elemento 4533  -- [ 35204 , 35 ] --  14
  procesando elemento 4539  -- [ 35740 , 33 ] --  9
  procesando elemento 4536  -- [ 34944 , 32 ] --  16
  procesando elemento 4532  -- [ 34799 , 34 ] --  4
  procesan

In [16]:
#encounters
encounters = sorted(encounters, key=itemgetter(0)) 
len(encounters)

4608

In [21]:
prob_probability = dict()

cont=1
for h in range(len(encounters)):
    if(len(encounters[h][1])!=0):
        map_infected=encounters[h][1].loc[:,['device_i','lon_i','lat_i','time_i']]
        map_noinfected=encounters[h][1].loc[:,['device_i','lon_i','lat_i','time_i','device_j','lon_j','lat_j','time_j','distance']]
                
        map_infected=map_infected.groupby( [ "device_i"] ).size().to_frame(name = 'count').reset_index()
        map_infected['prob']=[1.0]*len(map_infected)
        map_infected['lat_i']=[None]*len(map_infected)
        map_infected['lon_i']=[None]*len(map_infected)
        map_infected['time_i']=[None]*len(map_infected)
        map_infected['device_infectado']=[None]*len(map_infected)
        map_infected['lat_infectado']=[None]*len(map_infected)
        map_infected['lon_infectado']=[None]*len(map_infected)
        map_infected['time_infectado']=[None]*len(map_infected)
        map_infected = map_infected[["device_i", "lat_i", "lon_i","time_i","count","prob","device_infectado","lat_infectado","lon_infectado","time_infectado"]]
        map_infected.columns=["device","lat","lon","time","distance","prob","device_infectado","lat_infectado","lon_infectado","time_infectado"]
        users=map_noinfected['device_j'].unique()
        
        
        dict_probability=dict()
        probability_df=[]
        for u in users:
            try:
                dict_probability[u]=[prob_device[u]]
            except:
                dict_probability[u]=[0]
        user=[]
        probability=[]
        for e in range(len(encounters[h][1])):
            probcontagio=probaContagius(encounters[h][1]['lat_i'][e],encounters[h][1]['lon_i'][e],
                                        encounters[h][1]['lat_j'][e],encounters[h][1]['lon_j'][e],M)
            probability_df.append(probcontagio)
            dict_probability[encounters[h][1]['device_j'][e]].append(probcontagio)
        map_noinfected['prob']=probability_df  
        for d in dict_probability:
            for i in range(len(dict_probability[d])-1):
                if(i==0):
                    q=componeProbs(dict_probability[d][i],dict_probability[d][i+1])
                else:
                    q=componeProbs(q,dict_probability[d][i+1])
            user.append(d)
            probability.append(q)
            
        list_df=[]
        for i in range(len(user)):
            df=map_noinfected[map_noinfected['device_j']==user[i]].reset_index(drop=True)
            df['prob']=probability[i]
            list_df.append(df)
            
            prob_probability[user[i]] = probability[i]            
            
        map_noinfected=pd.concat(list_df)
        map_noinfected = map_noinfected[["device_j", "lat_j", "lon_j","time_j","distance","prob","device_i","lat_i","lon_i","time_i"]]
        map_noinfected.columns=["device","lat","lon","time","distance","prob","device_infectado","lat_infectado","lon_infectado","time_infectado"]
        data=pd.concat([map_infected,map_noinfected])
        data.to_csv(OUTPUTPATH+"eventos"+str(cont)+".csv",index=False,header=False)
        cont=cont+1
        
    else:
        data=pd.DataFrame(columns=["device","lat","lon","time","distance","prob","device_infectado","lat_infectado","lon_infectado","time_infectado"])
        data.to_csv(OUTPUTPATH+"eventos"+str(cont)+".csv",index=False,header=False)
        cont=cont+1
#data_probability=pd.DataFrame(prob.items(), columns=['user_id', 'Probability'])

print("Eventos guardados ...")

Eventos guardados ...


In [22]:
data_probability=pd.DataFrame(prob_probability.items(), columns=['Device_id', 'Probability'])
len(data_probability)

2462

In [23]:
usuario_id = []
UserDeviceDF = getUserDeviceID()
UserDeviceDic = dict(zip(UserDeviceDF.device_id, UserDeviceDF.user_id))
del UserDeviceDF

for item in data_probability['Device_id']:
    try:
        usuario_id.append(UserDeviceDic[item])
    except:
        usuario_id.append(None)
data_probability['user_id'] = usuario_id

In [24]:
data_probability.to_csv(OUTPUTPATH+"probability.csv",index=False)
data_probability.head()

,Device_id,Probability,user_id
0,099c144e-09c9-464b-a1a9-c18f741e021d,0.002674,fed8a8d82d2a781d2b26194834e15527591265c0000053...
1,377af371-b078-4b78-85d3-27fff599aeff,0.002670,1a831fa8b47a94b6eb60c5914cba923b9b79b05fa281e2...
2,f13ca397-98aa-47cc-b34c-d5d89341331e,0.002673,1bfb054e3536c39436659250729c3685081c7ec1397132...
3,eb6ce8cd-969a-457a-825b-beda56b3cc08,0.007996,0462086504b6727ccc96a7d3b62621c547fb402daffe1f...
4,457577b7-b42f-4408-8b63-d163c85f3d78,0.002674,efcb7833b2852ebb09ec72a95424a53948233c3034394e...
